# Gensim, word2vec

Gensim &ndash; это библиотека для тематического моделирования текстов. Один из компонентов в ней &ndash; питоновская обёртка для word2vec (который в оригинале был написан на C++).

Если gensim у вас не стоит, то ставим:

`pip install gensim`

In [1]:
import gensim

## Работа с моделью

Для каких-то своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь &ndash; http://rusvectores.org/ru/models

Скачаем модель для русского языка, созданную на основе НКРЯ. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте:

In [2]:
m = 'ruscorpora_rusvectores2.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

In [3]:
model.init_sims(replace=True)

Скажем, нам интересны такие слова (пример для русского языка):

In [4]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'студент_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели &ndash; она была натренирована на словах, аннотированных их частями речи (и лемматизированных).

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [5]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S 0.7165195941925049
месяц_S 0.631048858165741
вечер_S 0.5828738808631897
утро_S 0.5676207542419434
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224717378616333
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.57301926612854
ноченька_S 0.5582467317581177
рассвет_S 0.555358350276947
ночка_S 0.5351512432098389
полдень_S 0.5334426760673523
полночь_S 0.4786943197250366
день_S 0.478074848651886
сумерки_S 0.4390218257904053
фундерфун_S 0.4340825080871582


человек_S
[ 0.02013756 -0.02670704 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240307 -0.00025525 -0.14045683  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.49917873

Находим косинусную близость пары слов:

In [6]:
print(model.similarity('человек_S', 'обезьяна_S'))

0.238956092849


Найди лишнее!

In [7]:
print(model.doesnt_match('яблоко_S груша_S виноград_S банан_S картофель_S'.split()))

картофель_S


Реши пропорцию!

In [8]:
print(model.most_similar(positive=['пицца_S', 'россия_S'], negative=['италия_S'])[0][0])

плов_S


*Туториал основан на https://github.com/elmiram/2016learnpython/blob/master/word2vec.ipynb.*

## Домашнее задание

В качестве домашнего задания надо будет написать чатбота, который превращает все слова в сообщении в ближайшие синонимы.

Для этого нам понадобится сначала лемматизировать и разметить по частям речи слова при помощи пакета `pymystem3`.

Как и всё, что мы устанавливали до этого, `pymystem3` нужно установить либо через PyCharm, либо при помощи `pip install pymystem3`.

In [9]:
from pymystem3 import Mystem
m = Mystem() # инициализируем анализатор

Лемматизация текста:

In [10]:
m.lemmatize('Мама, МЫЛА красивую    "раму"!')

['мама', ', ', 'мыть', ' ', 'красивый', '    "', 'рама', '"', '!', '\n']

Как видно, Mystem сам умеет разбивать текст на токены и выделять отдельные слова.

Анализ текста:

In [11]:
m.analyze('Мама, МЫЛА    красивую "раму"!')

[{'analysis': [{'gr': 'S,жен,од=им,ед', 'lex': 'мама'}], 'text': 'Мама'},
 {'text': ', '},
 {'analysis': [{'gr': 'V,несов,пе=прош,ед,изъяв,жен', 'lex': 'мыть'}],
  'text': 'МЫЛА'},
 {'text': '    '},
 {'analysis': [{'gr': 'A=вин,ед,полн,жен', 'lex': 'красивый'}],
  'text': 'красивую'},
 {'text': ' "'},
 {'analysis': [{'gr': 'S,жен,неод=вин,ед', 'lex': 'рама'}], 'text': 'раму'},
 {'text': '"'},
 {'text': '!'},
 {'text': '\n'}]

Чтобы можно было применять к нашему тексту модель, нужно из массива, который получается при анализе, извлечь лексему ('lex') и частеречный тег ('gr', без дополнительных характеристик типа падежа, рода, числа и так далее), и записать в формате *[мама_S, мыть_V, красивый_A, рама_S]*.

Тогда чатбот должен будет выдать примерно следующее:

'папа_S помыть_V симпатичный_A створка_S'